Este notebook muestra un enfoque para manejar conjuntos de datos desbalanceados utilizando el conjunto de datos MNIST como ejemplo. Primero, se cuentan el número de muestras en cada clase para identificar las clases minoritarias. Luego, se realiza un sobre-muestreo (up-sampling) para igualar el número de muestras en todas las clases utilizando la función resample de sklearn.utils. A continuación, se define un nuevo conjunto de datos equilibrado BalancedMNISTDataset que utiliza los índices de las muestras sobre-muestreadas. Finalmente, los datos equilibrados se cargan utilizando DataLoader y se itera sobre ellos.

In [8]:
import torch
import torchvision
import torchvision.transforms as transforms
from sklearn.utils import resample
from torch.utils.data import DataLoader, Dataset


In [9]:
# Transformaciones de datos
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [10]:
# Descarga y carga del conjunto de datos MNIST
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)

In [11]:
# Dividir el conjunto de datos en clases
class_samples = [[] for _ in range(10)]
for idx, (image, label) in enumerate(trainset):
    class_samples[label].append(idx)

In [12]:
# Contar el número de muestras en cada clase
class_counts = [len(samples) for samples in class_samples]
print("Número de muestras por clase:", class_counts)

Número de muestras por clase: [5923, 6742, 5958, 6131, 5842, 5421, 5918, 6265, 5851, 5949]


In [13]:
# Rebalanceo de clases utilizando sobre-muestreo (up-sampling)
max_samples = max(class_counts)
resampled_samples = []
for samples in class_samples:
    if len(samples) < max_samples:
        resampled_samples.extend(resample(samples, replace=True, n_samples=max_samples))
    else:
        resampled_samples.extend(samples)

In [14]:
# Definir un nuevo conjunto de datos equilibrado
class BalancedMNISTDataset(Dataset):
    def __init__(self, dataset, resampled_samples):
        self.dataset = dataset
        self.resampled_samples = resampled_samples

    def __getitem__(self, index):
        original_index = self.resampled_samples[index]
        return self.dataset[original_index]

    def __len__(self):
        return len(self.resampled_samples)


In [15]:
# Crear un nuevo conjunto de datos equilibrado
balanced_trainset = BalancedMNISTDataset(trainset, resampled_samples)

# Cargar los datos equilibrados utilizando DataLoader
batch_size = 64
trainloader = DataLoader(balanced_trainset, batch_size=batch_size, shuffle=True)

In [16]:
# Iterar sobre los datos equilibrados
for images, labels in trainloader:
    # Aquí puedes realizar el entrenamiento de tu modelo
    print("Tamaño del lote:", len(labels))
    break  # Detener después de mostrar un lote


Tamaño del lote: 64


Es importante destacar que este es solo un ejemplo básico de cómo manejar conjuntos de datos desbalanceados y que existen otros métodos y enfoques más sofisticados para tratar este problema, como sub-muestreo, técnicas de generación de muestras sintéticas, el uso de pesos de clase, entre otros. Los métodos adecuados pueden variar según el problema y es recomendable explorar más opciones según tu caso específico.

**Ejercicio: Manejo de conjuntos de datos desbalanceados en PyTorch**

1. Descarga un conjunto de datos de clasificación desbalanceado, como el conjunto de datos "Credit Card Fraud Detection" disponible en Kaggle: [Credit Card Fraud Detection Dataset](https://www.kaggle.com/mlg-ulb/creditcardfraud)

2. Divide el conjunto de datos descargado en conjuntos de entrenamiento y prueba.

3. Calcula la distribución de clases en el conjunto de entrenamiento y muestra el número de instancias para cada clase. Observa el desequilibrio de clases en el conjunto de datos.

4. Implementa una estrategia para manejar el desequilibrio de clases en el conjunto de entrenamiento. Puedes utilizar alguna técnica de remuestreo, como sobremuestreo (oversampling) o submuestreo (undersampling), o una combinación de ambas. Puedes utilizar la biblioteca `imbalanced-learn` para implementar estas técnicas.

5. Crea una clase personalizada llamada `MiConjuntoDeDatosBalanceado` que herede de `torch.utils.data.Dataset`. Esta clase debe cargar el conjunto de datos de entrenamiento después de aplicar la estrategia de manejo de desequilibrio de clases.

6. Implementa los métodos `__len__` y `__getitem__` en la clase `MiConjuntoDeDatosBalanceado` para obtener la longitud del conjunto de datos y obtener elementos individuales.

7. Crea una instancia de `MiConjuntoDeDatosBalanceado` utilizando el conjunto de datos de entrenamiento balanceado.

8. Utiliza la clase `torch.utils.data.DataLoader` para crear un dataloader utilizando el conjunto de datos balanceado. Asegúrate de especificar un tamaño de lote (`batch_size`) adecuado.

9. Entrena un modelo de clasificación utilizando el dataloader creado y evalúa su desempeño en el conjunto de prueba desbalanceado.

**Pista**: Puedes utilizar la biblioteca `scikit-learn` y `imbalanced-learn` para realizar la división del conjunto de datos, calcular la distribución de clases y aplicar técnicas de manejo de desequilibrio de clases.

In [7]:
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
import pandas as pd

# Paso 1: Cargar el conjunto de datos "Fraudulent Credit Card Transactions" utilizando pandas
df = pd.read_csv("creditcard.csv")

# Paso 2: Dividir el conjunto de datos en características (X) y etiquetas (y)
X = df.drop(["Class"], axis=1).values
y = df["Class"].values

# Paso 3: Dividir el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Paso 4: Aplicar una estrategia para manejar el desequilibrio de clases en el conjunto de entrenamiento
undersampler = RandomUnderSampler(sampling_strategy='majority')
X_train_resampled, y_train_resampled = undersampler.fit_resample(X_train, y_train)

# Paso 5: Crear una clase personalizada para el conjunto de datos balanceado
class MiConjuntoDeDatosBalanceado(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Paso 6: Crear una instancia del conjunto de datos balanceado
mi_conjunto_de_datos = MiConjuntoDeDatosBalanceado(X_train_resampled, y_train_resampled)

# Paso 7: Crear un dataloader utilizando el conjunto de datos balanceado
batch_size = 32
dataloader = DataLoader(mi_conjunto_de_datos, batch_size=batch_size, shuffle=True)

# Paso 8: Entrenar un modelo de clasificación utilizando el dataloader y evaluar su desempeño en el conjunto de prueba desbalanceado
# (código para entrenar y evaluar el modelo)
